In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan import DATWGAN

import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'
name = 'LINEAR'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,3.93477,2,3,3,detached,6,0,30,1,4,PTE,20.166667
1,drive,2,SHOPPING,0.31557,3,3,3,detached,7,0,54,1,5,FTE,17.500000
2,drive,2,SHOPPING,0.28349,1,1,0,detached,3,0,80,1,3,PTE,9.333333
3,drive,2,OTHER,0.69417,2,2,0,detached,5,1,42,1,5,FTE,13.783333
4,passenger,1,SHOPPING,4.30666,2,2,1,detached,4,0,32,0,3,Unemployed,11.566667


In [7]:
continuous_columns = ["distance", "age", "departure_time"]

In [8]:
# personalised graph
graph = nx.DiGraph()

if name in ['FULLDAG', 'TRANSRED']:
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
    
    if name is 'TRANSRED':
        graph = nx.transitive_reduction(graph)
elif name is 'LINEAR':
    list_ = []
    for i in range(len(df.columns)-1):
        list_.append((df.columns[i], df.columns[i+1]))
    graph.add_edges_from(list_)
elif name is 'NOLINKS':
    for c in df.columns:
        graph.add_node(c)
elif name is 'CHOICE':
    list_ = []
    for c in df.columns:
        if c == 'choice':
            pass
        else:
            list_.append((c, 'choice'))
    graph.add_edges_from(list_)
else:
    print("PROBLEM")

In [9]:
name = 'BOB05'

In [10]:
output_folder = '../output/' + dataset + '/{}/'.format(name)

In [11]:
datgan = DATWGAN(continuous_columns, max_epoch=1000, batch_size=500, output=output_folder, gpu=0)

In [12]:
test = datgan.fit(df, graph)

[1110 21:59:42 @DATSGAN.py:166] Preprocessing the data!
[1110 21:59:42 @data.py:383] Encoding categorical variable "choice"...
[1110 21:59:42 @data.py:383] Encoding categorical variable "travel_dow"...
[1110 21:59:42 @data.py:383] Encoding categorical variable "trip_purpose"...
[1110 21:59:42 @data.py:370] Encoding continuous variable "distance"...
[1110 21:59:42 @data.py:239]   Fitting model with 10 components
[1110 21:59:51 @data.py:266]   Predictions were done on 7 components => Fit with 7 components!
[1110 21:59:58 @data.py:269]   Some weights are too small =>  => Fit with 6 components!
[1110 22:00:04 @data.py:271]   Predictions were done on 6 components => FINISHED!
[1110 22:00:04 @data.py:274]   Train VGM with full data
[1110 22:00:10 @data.py:383] Encoding categorical variable "hh_vehicles"...
[1110 22:00:10 @data.py:383] Encoding categorical variable "hh_size"...
[1110 22:00:10 @data.py:383] Encoding categorical variable "hh_bikes"...
[1110 22:00:10 @data.py:383] Encoding categ

[1110 22:00:55 @registry.py:126] gen/LSTM/trip_purpose/FC_prob input: [500, 50]
[1110 22:00:55 @registry.py:134] gen/LSTM/trip_purpose/FC_prob output: [500, 7]
[1110 22:00:55 @registry.py:126] gen/LSTM/trip_purpose/FC_input input: [500, 7]
[1110 22:00:55 @registry.py:134] gen/LSTM/trip_purpose/FC_input output: [500, 100]
[1110 22:00:55 @DATSGANModel.py:207] Creating cell for distance (in-edges: 1; ancestors: 3)
[1110 22:00:55 @registry.py:126] gen/LSTM/distance/FC input: [500, 100]
[1110 22:00:56 @registry.py:134] gen/LSTM/distance/FC output: [500, 50]
[1110 22:00:56 @registry.py:126] gen/LSTM/distance/FC_val input: [500, 50]
[1110 22:00:56 @registry.py:134] gen/LSTM/distance/FC_val output: [500, 6]
[1110 22:00:56 @registry.py:126] gen/LSTM/distance/FC_prob input: [500, 50]
[1110 22:00:56 @registry.py:134] gen/LSTM/distance/FC_prob output: [500, 6]
[1110 22:00:56 @registry.py:126] gen/LSTM/distance/FC_input input: [500, 12]
[1110 22:00:56 @registry.py:134] gen/LSTM/distance/FC_input ou



[1110 22:00:56 @registry.py:126] discrim/DISCR_FC_0/FC input: [500, 107]
[1110 22:00:56 @registry.py:134] discrim/DISCR_FC_0/FC output: [500, 100]
[1110 22:00:56 @registry.py:126] discrim/DISCR_FC_0/FC_DIVERSITY input: [500, 100]
[1110 22:00:56 @registry.py:134] discrim/DISCR_FC_0/FC_DIVERSITY output: [500, 100]


[1110 22:00:56 @registry.py:126] discrim/DISCR_FC_TOP input: [500, 110]
[1110 22:00:56 @registry.py:134] discrim/DISCR_FC_TOP output: [500, 1]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1110 22:00:59 @logger.py:90] Argv: D:\Users\glede\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py -f C:\Users\glede\AppData\Roaming\jupyter\runtime\kernel-a968385c-2518-4e98-a789-7f0f785c66aa.json



[1110 22:00:59 @model_utils.py:67] List of Trainable Variables: 
name                                         shape         #elements
-------------------------------------------  ----------  -----------
gen/

[1110 22:00:59 @base.py:209] Setup callbacks graph ...







[1110 22:00:59 @utils.py:15] Clip discrim/DISCR_FC_0/FC/W

[1110 22:00:59 @utils.py:15] Clip discrim/DISCR_FC_0/FC/b
[1110 22:00:59 @utils.py:15] Clip discrim/DISCR_FC_0/FC_DIVERSITY/W
[1110 22:00:59 @utils.py:15] Clip discrim/DISCR_FC_0/FC_DIVERSITY/b
[1110 22:00:59 @utils.py:15] Clip discrim/DISCR_FC_0/BN/beta
[1110 22:00:59 @utils.py:15] Clip discrim/DISCR_FC_TOP/W
[1110 22:00:59 @utils.py:15] Clip discrim/DISCR_FC_TOP/b
[1110 22:00:59 @summary.py:46] [MovingAverageSummary] 3 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[1110 22:00:59 @summary.py:93] Summarizing collection 'summaries' of size 4.

[1110 22:00:59 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.

[1110 22:01:00 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[1110 22:01:01 @base.py:236] Initializing the session ...
[1110 22:01:01 

 18%|###############1                                                                      |3/17[00:02<00:10, 1.39it/s]

...


100%|#####################################################################################|17/17[00:00<00:00,37.87it/s]

[1110 22:10:44 @base.py:285] Epoch 999 (global_step 25474) finished, time:0.449 second.
[1110 22:10:44 @saver.py:79] Model saved to ../output/Chicago/BOB05/model\model-25474.
[1110 22:10:44 @monitor.py:467] GAN_loss/d_loss: -0.06159
[1110 22:10:44 @monitor.py:467] GAN_loss/g_loss: -0.085338
[1110 22:10:44 @monitor.py:467] GAN_loss/kl_div: 0.043962
[1110 22:10:44 @monitor.py:467] QueueInput/queue_size: 50
[1110 22:10:44 @base.py:275] Start Epoch 1000 ...



100%|#####################################################################################|17/17[00:00<00:00,39.28it/s]

[1110 22:10:45 @base.py:285] Epoch 1000 (global_step 25499) finished, time:0.433 second.
[1110 22:10:45 @saver.py:79] Model saved to ../output/Chicago/BOB05/model\model-25499.
[1110 22:10:45 @monitor.py:467] GAN_loss/d_loss: -0.060484
[1110 22:10:45 @monitor.py:467] GAN_loss/g_loss: -0.085711
[1110 22:10:45 @monitor.py:467] GAN_loss/kl_div: 0.043387
[1110 22:10:45 @monitor.py:467] QueueInput/queue_size: 50
[1110 22:10:45 @base.py:289] Training has finished!

[1110 22:10:45 @DATSGANModel.py:207] Creating cell for choice (in-edges: 0; ancestors: 0)


[1110 22:10:45 @DATSGANModel.py:207] Creating cell for travel_dow (in-edges: 1; ancestors: 1)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for trip_purpose (in-edges: 1; ancestors: 2)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for distance (in-edges: 1; ancestors: 3)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for hh_vehicles (in-edges: 1; ancestors: 4)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for hh_size (in-edges: 1; ancestors: 5)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for hh_bikes (in-edges: 1; ancestors: 6)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for hh_descr (in-edges: 1; ancestors: 7)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for hh_income (in-edges: 1; ancestors: 8)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for gender (in-edges: 1; ancestors: 9)
[1110 22:10:45 @DATSGANModel.py:207] Creating cell for age (in-edges: 1; ancestors: 10)
[1110 22:10:46 @DATSGANModel.py:207] Creating cell for license (in-edges: 1; 

In [13]:
datgan.save('trained', force=True)

[1110 22:11:00 @DATSGAN.py:364] Model saved successfully.


In [14]:
beepy.beep(6)